In [50]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
import os
# Import the API key.
from config import g_key
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [12]:
# Store the CSV you saved created in part one into a DataFrame.
CityDataToLoad = os.path.join("data","WeatherPy_database.csv")
city_data_df = pd.read_csv(CityDataToLoad)
city_data_df = city_data_df.set_index("City_ID")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (Inches),Snow (Inches),Date
City_ID,,,,,,,,,,,,
0,Mataura,NZ,-46.19,168.86,41.38,91,100,5.75,overcast clouds,0.0,0.0,2020-07-09 18:44:51
1,Hambantota,LK,6.12,81.12,78.80,100,40,3.36,scattered clouds,0.0,0.0,2020-07-09 18:44:51
2,Laguna,US,38.42,-121.42,84.99,43,0,1.01,clear sky,0.0,0.0,2020-07-09 18:44:51
3,Lebu,CL,-37.62,-73.65,53.69,93,100,24.94,light rain,0.0,0.0,2020-07-09 18:44:51
4,Mahebourg,MU,-20.41,57.70,71.60,68,75,17.22,light intensity shower rain,0.0,0.0,2020-07-09 18:44:52


In [13]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [29]:
#Gather User Parameters
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
raining = ""
snowing = ""
while raining not in ("yes","no"):
    raining = input("Do you want it to be raining? (yes/no) ")
while snowing not in ("yes","no"):
    snowing = input("Do you want it to be snowing? (yes/no) ")

In [30]:
# Filter the dataset to find the cities that fit the criteria.
if raining == "yes" and snowing == "yes":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Rain (Inches)"] > 0) & \
                                       (city_data_df["Snow (Inches)"] > 0)]
elif raining == "yes" and snowing == "no":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Rain (Inches)"] > 0) & \
                                       (city_data_df["Snow (Inches)"] == 0)]
elif raining == "no" and snowing == "yes":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Rain (Inches)"] == 0) & \
                                       (city_data_df["Snow (Inches)"] > 0)]
else:
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Rain (Inches)"] == 0) & \
                                       (city_data_df["Snow (Inches)"] == 0)]
preferred_cities_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (Inches),Snow (Inches),Date
City_ID,,,,,,,,,,,,
1,Hambantota,LK,6.12,81.12,78.80,100,40,3.36,scattered clouds,0.0,0.0,2020-07-09 18:44:51
5,Avarua,CK,-21.21,-159.78,75.20,73,20,10.29,few clouds,0.0,0.0,2020-07-09 18:44:52
12,Birin,NP,26.89,88.07,78.80,94,75,3.36,broken clouds,0.0,0.0,2020-07-09 18:35:25
13,Ponta Do Sol,PT,32.67,-17.10,75.20,69,20,16.11,few clouds,0.0,0.0,2020-07-09 18:44:53
22,Sibolga,ID,1.74,98.78,78.03,71,100,2.24,overcast clouds,0.0,0.0,2020-07-09 18:44:55
24,Kokopo,PG,-4.35,152.26,77.07,79,66,5.93,broken clouds,0.0,0.0,2020-07-09 18:44:55
31,Mahon,ES,39.89,4.27,77.00,69,8,9.17,clear sky,0.0,0.0,2020-07-09 18:44:57
32,Balkhash,KZ,46.84,74.98,75.47,44,17,16.91,few clouds,0.0,0.0,2020-07-09 18:44:57
36,Kloulklubed,PW,7.04,134.26,77.00,94,90,3.36,overcast clouds,0.0,0.0,2020-07-09 18:44:58


In [34]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp","Current Description","Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""

In [35]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [36]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [37]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
City_ID,,,,,,,
1,Hambantota,LK,78.80,scattered clouds,6.12,81.12,Bungalow 63
5,Avarua,CK,75.20,few clouds,-21.21,-159.78,Paradise Inn
12,Birin,NP,78.80,broken clouds,26.89,88.07,HOTEL SURYODAYA
13,Ponta Do Sol,PT,75.20,few clouds,32.67,-17.10,Hotel do Campo
22,Sibolga,ID,78.03,overcast clouds,1.74,98.78,Hotel Syariah CN Darussalam
24,Kokopo,PG,77.07,broken clouds,-4.35,152.26,Kokopo Beach Bungalow Resort
31,Mahon,ES,77.00,clear sky,39.89,4.27,ARTIEM Capri
32,Balkhash,KZ,75.47,few clouds,46.84,74.98,Family
36,Kloulklubed,PW,77.00,overcast clouds,7.04,134.26,Storyboard Beach Resort


In [44]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [55]:
# Map the Vacation spots with a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [48]:
# Create the output file (CSV).
output_data_file = os.path.join("data","WeatherPy_vacation.csv")
# Export the City_Data into a CSV.
preferred_cities_df.to_csv(output_data_file, index_label="City_ID")